In [7]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
from importlib import reload

import Tree
reload(Tree)
from Tree import *

import forecast_prophet
reload(forecast_prophet)
from forecast_prophet import *

import forecast_arima
reload(forecast_arima)
from forecast_arima import *

import utils
reload(utils)
from utils import *

import main
reload(main)
from main import *

path='c:\\Users\\31683\\Desktop\\data\\M5'
Y_path=path+f"\\sales_train_validation.pkl"  # to data file 
Y_path_eval=path+f"\\sales_train_evaluation.pkl" 
X_path_val_eval=path+f"\\prices_train_val_eval.pkl"  # to data file
df_holidays=pd.read_csv(path+f"\\holidays.csv")

df_data=pd.read_pickle(Y_path)


# mX  = get_mX(X_path_val_eval)

c:\Users\31683\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## original

In [163]:
import numpy as np
from numpy import linalg as LA
import gurobipy as gp
from gurobipy import GRB, quicksum
# Gurobi Optimizer version 10.0.1 build v10.0.1rc0
# import math

def eglasso1(Y, Y_hat, S, l1 = 0, m = None, M = None, weight = True, TimeLimit = 0, LogToConsole = 0, OutputFlag = 0):
    """
    Solve the SOCP problem: min_{G} 0.5 * 1/N * ||vec(Y) - np.kron(S, Y) vec(G')||_2^2 + l1 * sum_{j}(w_{j} ||G_{.j}||_2)
                          

    Parameters
    ----------
    Y : np.array
        N x n numpy array of true values.
    Y_hat: np.array
        N x n numpy array of base forecasts.
    S : np.array
        n x nb numpy array describing the hierarchy structure.
    l1 : float, optional
        lagrange multiplier.
    weight: int
        Enables weighted group-lasso when it is True.
    TimeLimit: float, optional
        set a timeout for gurobi.
    LogToConsole: int, optional
        Enables or disables console logging. Use OutputFlag to shut off all logging.
    OutputFlag: int, optional
        Enables or disables solver output. Use LogFile and LogToConsole for finer-grain control. Setting OutputFlag to 0 is equivalent to setting LogFile to "" and LogToConsole to 0.
        
        
    Returns
    -------
    G, Z, obj

    """
    
    N = Y.shape[0]
    n = S.shape[0]
    nb = S.shape[1]
    p = nb * n
    Nn = N * n
    
    y = Y.T.reshape(Nn) # vec(Y)
    
    """ OLS solution """
    W = np.identity(n)
    inv_W = np.linalg.inv(W)
    R = S.T @ inv_W
    G_ols = np.linalg.inv(R @ S) @ R
    
    """ Penalty factor """
    if weight:
        w = 1/LA.norm(G_ols, axis=0)
    else:
        w = np.repeat(1, n)
    
    # w = w/w.sum() # normalize weight vector to sum to 1 
    
    """ Bound """
    if m is None:
        m = np.amax(abs(G_ols)) + 1
    if M is None:
        M = nb
    emax = np.amax(abs(y))
    
    """ SUPPRESS ALL OUTPUT """
    env = gp.Env(empty=True)
    env.setParam("OutputFlag",OutputFlag)
    env.start()
    
    """ EGLASSO MODEL """
    model = gp.Model('EGLASSO', env=env) # the optimization model
    
    """ PARAMETERS """
    # G matrix: vec(G')
    G = model.addMVar(shape=(p, ), vtype=GRB.CONTINUOUS,
                      ub=np.repeat(m, p), lb=np.repeat(-m, p))
    # Error
    E = model.addMVar(shape=(Nn, ), vtype=GRB.CONTINUOUS,
                      ub=np.repeat(emax, Nn), lb=np.repeat(-emax, Nn))
    # Auxiliary variables for l2 norm
    AUX = model.addMVar(shape=(n, ), vtype=GRB.CONTINUOUS,
                        ub=GRB.INFINITY, lb=np.repeat(0, n))
    model.update()

    """ OBJECTIVE """
    model.setObjective(0.5 * 1/N * E.T @ E + l1 * quicksum(w * AUX), GRB.MINIMIZE)

    """ CONSTRAINTS """
    model.addConstr(y == E + np.kron(S, Y_hat) @ G)
    # SOC constraints
    for j in range(n):
        model.addGenConstrNorm(AUX[j], G[n * np.arange(nb) + j], 1)
    model.update()
    
    """ OUTPUT THE MODEL TO A FILE """ 
    # model.write("myfile.lp")
    
    """ OPTIMIZE """
    model.Params.OutputFlag = OutputFlag
    model.Params.LogToConsole = LogToConsole
    if p > 1000:
        model.Params.NumericFocus = 1
        model.Params.OptimalityTol = 1e-4
        model.Params.FeasibilityTol = 1e-4
        model.Params.BarConvTol = 1e-4
        model.Params.BarQCPConvTol = 1e-4
    if TimeLimit > 0:
        model.params.TimeLimit = TimeLimit
    model.optimize()
    # model.Params.Threads = 1
    
    g = G.X
    G = g.reshape(nb, n)
    Z = 1 - (~(abs(G) > 1e-5).any(axis=0))*1
    obj = model.objval
    
    return G, Z, obj

## modified

In [ ]:
import numpy as np
from numpy import linalg as LA
import gurobipy as gp
from gurobipy import GRB, quicksum

def eglasso2(Y, Y_hat, S, l1 = 0, m = None, M = None, weight = True, TimeLimit = 0, LogToConsole = 0, OutputFlag = 0):
    """
    Solve the SOCP problem: min_{G} 0.5 * 1/N * ||vec(Y) - np.kron(S, Y) vec(G')||_2^2 + l1 * sum_{j}(w_{j} ||G_{.j}||_2)
                          

    Parameters
    ----------
    Y : np.array
        T x n numpy array of true values.  220
    Y_hat: np.array
        T x n numpy array of base forecasts.  
    S : np.array
        n x nb numpy array describing the hierarchy structure. 8X7
    l1 : float, optional
        lagrange multiplier.
    weight: int
        Enables weighted group-lasso when it is True.
    TimeLimit: float, optional
        set a timeout for gurobi.
    LogToConsole: int, optional
        Enables or disables console logging. Use OutputFlag to shut off all logging.
    OutputFlag: int, optional
        Enables or disables solver output. Use LogFile and LogToConsole for finer-grain control. Setting OutputFlag to 0 is equivalent to setting LogFile to "" and LogToConsole to 0.
        
        
    Returns
    -------
    G, Z, obj

    """
    
    N = Y.shape[0]   #220
    n = S.shape[0]   #8
    nb = S.shape[1]  #7
    p = nb * n       #56
    Nn = N * n       #1760
    
    y = Y.T.reshape(Nn) # vec(Y)
    
    """ OLS solution """
    W = np.identity(n)
    inv_W = np.linalg.inv(W)
    R = S.T @ inv_W
    G_ols = np.linalg.inv(R @ S) @ R
    
    """ Penalty factor """
    if weight:
        w[0] = 1/LA.norm(G_ols[:,0], axis=0)
        w[1] = 1/LA.norm(G_ols[:,1:])
        
    else:
        w = np.repeat(1, 2)
    
    # # w = w/w.sum() # normalize weight vector to sum to 1 
    
    """ Bound """
    if m is None:
        m = np.amax(abs(G_ols)) + 1
    if M is None:
        M = nb
    emax = np.amax(abs(y))
    
    """ SUPPRESS ALL OUTPUT """
    env = gp.Env(empty=True)
    env.setParam("OutputFlag",OutputFlag)
    env.start()
    
    """ EGLASSO MODEL """
    model = gp.Model('EGLASSO', env=env) # the optimization model
    
    """ PARAMETERS """
    # G matrix: vec(G')
    G = model.addMVar(shape=(p, ), vtype=GRB.CONTINUOUS,
                      ub=np.repeat(m, p),  lb=np.repeat(-m, p))
    # Error
    E = model.addMVar(shape=(Nn, ), vtype=GRB.CONTINUOUS,
                      ub=np.repeat(emax, Nn), lb=np.repeat(-emax, Nn))
    # Auxiliary variables for l2 norm
    
    
    # AUX = model.addMVar(shape=(n, ), vtype=GRB.CONTINUOUS,
    #                     ub=GRB.INFINITY, lb=np.repeat(0, n))
    AUX = model.addMVar(shape=(2,), vtype=GRB.CONTINUOUS,ub=GRB.INFINITY, lb=np.repeat(0,2))
    model.update()

    """ OBJECTIVE """
    # model.setObjective(0.5 * 1/N * E.T @ E + l1 * quicksum(w * AUX), GRB.MINIMIZE)
    model.setObjective(0.5 * 1/N * E.T @ E + l1 * quicksum(w * AUX), GRB.MINIMIZE)

    """ CONSTRAINTS """
    model.addConstr(y == E + np.kron(S, Y_hat) @ G)
    # SOC constraints
    # for j in range(n):
    #     model.addGenConstrNorm(AUX[j], G[n * np.arange(nb) + j], 2)
    model.addGenConstrNorm(AUX[0], G[np.arange(nb)], 2)  # First column of G
    other_indices = np.hstack([n * np.arange(nb) + j for j in range(1, n)])  # Columns 2 to 7
    model.addGenConstrNorm(AUX[1], G[other_indices], 2)
    model.update()
    
    """ OUTPUT THE MODEL TO A FILE """ 
    # model.write("myfile.lp")
    
    """ OPTIMIZE """
    model.Params.OutputFlag = OutputFlag
    model.Params.LogToConsole = LogToConsole
    if p > 1000:
        model.Params.NumericFocus = 1
        model.Params.OptimalityTol = 1e-4
        model.Params.FeasibilityTol = 1e-4
        model.Params.BarConvTol = 1e-4
        model.Params.BarQCPConvTol = 1e-4
    if TimeLimit > 0:
        model.params.TimeLimit = TimeLimit
    model.optimize()
    # model.Params.Threads = 1
    
    g = G.X
    G = g.reshape(nb, n)
    Z = 1 - (~(abs(G) > 1e-5).any(axis=0))*1
    obj = model.objval
    
    return G, Z,  obj

In [140]:
leaf=100
transform='log'

tree_spatial=Tree( dfData= df_data , sType='spatial') 
df_data_leaf=pd.DataFrame(data=tree_spatial.mY[leaf,:-365], index=tree_spatial.date_time_index[:-365] , columns=['y'])

tree_temporal=Tree(dfData=df_data_leaf, sType='temporal', iLeaf=leaf)
tree_temporal.forecast_prophet(iOoS=7,sTransform=transform, mX=None,dfHolidays=df_holidays)



c:\Users\31683\Desktop\Hierarchical-Forecasting\Tree.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[level]=None


In [165]:
P, Z , obj = eglasso1(Y=tree_temporal.mY.T, Y_hat=tree_temporal.mYhatIS.T, S=tree_temporal.mS , l1=1000)
print(np.round(P,2))

[[ 0.11 -0.01 -0.08  0.06  0.22  0.   -0.    0.  ]
 [ 0.12 -0.02 -0.08  0.08  0.13  0.   -0.    0.  ]
 [ 0.15 -0.14 -0.18  0.    0.21  0.   -0.    0.  ]
 [ 0.14 -0.17 -0.07  0.02  0.17  0.   -0.    0.  ]
 [ 0.14 -0.08 -0.    0.08  0.13  0.   -0.    0.  ]
 [ 0.15 -0.13 -0.11  0.17  0.21  0.   -0.    0.  ]
 [ 0.12 -0.25 -0.    0.11  0.23  0.   -0.   -0.  ]]


In [169]:
P, Z , obj = eglasso2(Y=tree_temporal.mY.T, Y_hat=tree_temporal.mYhatIS.T, S=tree_temporal.mS , l1=1000)
print(np.round(P,2))


[[ 0.13 -0.   -0.    0.    0.    0.    0.    0.  ]
 [ 0.12 -0.02 -0.11  0.08  0.16  0.   -0.    0.  ]
 [ 0.15 -0.15 -0.2   0.    0.24  0.   -0.    0.  ]
 [ 0.14 -0.18 -0.1   0.02  0.2   0.   -0.   -0.  ]
 [ 0.14 -0.09 -0.    0.08  0.15  0.   -0.    0.  ]
 [ 0.14 -0.14 -0.14  0.17  0.23  0.   -0.    0.  ]
 [ 0.12 -0.27 -0.    0.11  0.26  0.   -0.    0.  ]]
